## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210210.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,475110,36300,34500,545910,401714,0.735861,159969,2021-02-09
1,Aragón,81045,6300,5400,92745,80211,0.864855,29343,2021-02-09
2,Asturias,83975,6300,4200,94475,75118,0.795110,32296,2021-02-09
3,Baleares,42480,3200,4800,50480,37685,0.746533,14564,2021-02-09
4,Canarias,93570,7000,8900,109470,86467,0.789869,29967,2021-02-09
5,Cantabria,39345,2900,2400,44645,36320,0.813529,14393,2021-02-09
6,Castilla y Leon,177555,13500,9800,200855,168030,0.836574,69345,2021-02-09
7,Castilla La Mancha,120375,9200,8300,137875,109636,0.795184,35531,2021-02-09
8,Cataluña,410120,31400,31800,473320,340729,0.719870,138400,2021-02-09
9,C. Valenciana,207280,15700,20600,243580,201983,0.829227,89522,2021-02-09


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,475110,36300,34500,545910,401714,0.735861,159969,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,81045,6300,5400,92745,80211,0.864855,29343,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,83975,6300,4200,94475,75118,0.795110,32296,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,42480,3200,4800,50480,37685,0.746533,14564,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,93570,7000,8900,109470,86467,0.789869,29967,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,39345,2900,2400,44645,36320,0.813529,14393,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,177555,13500,9800,200855,168030,0.836574,69345,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,120375,9200,8300,137875,109636,0.795184,35531,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,410120,31400,31800,473320,340729,0.719870,138400,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,207280,15700,20600,243580,201983,0.829227,89522,9/2/21,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [14]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [15]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,10/2/21,Andalucía,475110,36300,34500,545910,401714,0.735861,159969,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,10/2/21,Aragón,81045,6300,5400,92745,80211,0.864855,29343,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,10/2/21,Asturias,83975,6300,4200,94475,75118,0.795110,32296,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,10/2/21,Baleares,42480,3200,4800,50480,37685,0.746533,14564,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,10/2/21,Canarias,93570,7000,8900,109470,86467,0.789869,29967,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,10/2/21,Cantabria,39345,2900,2400,44645,36320,0.813529,14393,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,10/2/21,Castilla y Leon,177555,13500,9800,200855,168030,0.836574,69345,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,10/2/21,Castilla La Mancha,120375,9200,8300,137875,109636,0.795184,35531,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,10/2/21,Cataluña,410120,31400,31800,473320,340729,0.719870,138400,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,10/2/21,C. Valenciana,207280,15700,20600,243580,201983,0.829227,89522,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [16]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [17]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [18]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,9/2/21,Navarra,33495.0,1300.0,2700.0,37495,35585,0.949060,12509.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
516,9/2/21,País Vasco,91080.0,3600.0,9100.0,103780,75634,0.728792,26299.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
517,9/2/21,Ceuta,1980.0,200.0,400.0,2580,2218,0.859690,964.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
518,9/2/21,Melilla,1980.0,200.0,400.0,2580,2399,0.929845,1080.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [19]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [20]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [21]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,10/2/21,Navarra,39345.0,2900.0,2700.0,44945,36677,0.816042,13286.0,NaN,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
536,10/2/21,País Vasco,107460.0,7900.0,9100.0,124460,75634,0.607697,26299.0,NaN,9/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
537,10/2/21,Ceuta,4320.0,400.0,400.0,5120,2277,0.444727,963.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
538,10/2/21,Melilla,3150.0,400.0,400.0,3950,2399,0.607342,1080.0,NaN,8/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [23]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [24]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)